In [1]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import numpy as np
import pandas as pd

In [11]:
df=pd.read_csv('SMSspamcollection.tsv',sep='\t')

In [12]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [13]:
df.describe()

,length,punct
count,5572.000000,5572.000000
mean,80.489950,4.177495
std,59.942907,4.623919
min,2.000000,0.000000
25%,36.000000,2.000000
50%,62.000000,3.000000
75%,122.000000,6.000000
max,910.000000,133.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
 2   length   5572 non-null   int64 
 3   punct    5572 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 174.3+ KB


# Checking null values


In [15]:
df.isnull().sum() # no null values

label      0
message    0
length     0
punct      0
dtype: int64

# Counting the Labels

In [16]:
df['label'].value_counts() 

label
ham     4825
spam     747
Name: count, dtype: int64

# Splitting data into test and train set

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = df['message']
y = df['label']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer

Used to transform a given text into a vector on the basis of the frequency(count) of each word that occurs in the entire text

In [49]:
count_vect=CountVectorizer()

In [50]:
X[:2]

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
Name: message, dtype: object

# Fit the data to vectorizer

1] Build vocab  2] Count number of words

In [23]:
# Fit the vectorizer to the data  (build a vocab, count the number of words)
count_vect.fit(X_train)
X_train_counts=count_vect.transform(X_train)

In [24]:
X_train_counts

<3900x7263 sparse matrix of type '<class 'numpy.int64'>'
	with 52150 stored elements in Compressed Sparse Row format>

In [25]:
X_train.shape

(3900,)

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(3900, 7263)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer

The TfidfVectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allow you to encode new documents.

In [29]:
vectorizer=TfidfVectorizer()

In [30]:
X_train_tfidf=vectorizer.fit_transform(X_train)

In [31]:
from sklearn.svm import LinearSVC

# LinearSVC (support vector classifier)

The objective of a Linear SVC (Support Vector Classifier) is to fit to the data you provide, returning a "best fit" hyperplane that divides, or categorizes, your data

In [34]:
clf=LinearSVC()

In [35]:
clf.fit(X_train_tfidf,y_train)

C:\Users\RINGROW\AppData\Roaming\Python\Python311\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [36]:
from sklearn.pipeline import Pipeline

In [37]:
text_clf=Pipeline([('tfidf', TfidfVectorizer()), ('clf',LinearSVC())])

In [38]:
text_clf.fit(X_train, y_train)

C:\Users\RINGROW\AppData\Roaming\Python\Python311\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [39]:
predictions=text_clf.predict(X_test)

In [40]:
from sklearn.metrics import confusion_matrix, classification_report

In [41]:
print(confusion_matrix(y_test, predictions))

[[1445    3]
 [  10  214]]


In [42]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1448
        spam       0.99      0.96      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.98      0.98      1672
weighted avg       0.99      0.99      0.99      1672



In [44]:
from sklearn import metrics
metrics.accuracy_score(y_test,predictions)

0.9922248803827751

In [47]:
text_clf.predict(["Hello, my Name is Suhani?"])

array(['ham'], dtype=object)

In [48]:
text_clf.predict(["Free car wash with mobile phone"])

array(['spam'], dtype=object)